In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "NF-UNSW-NB15-v3.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
0    2151027
1      91904
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
# ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [11]:
# ADDED
test_data.Attack.unique()
##

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

IPV4_SRC_ADDR  IPV4_DST_ADDR  PROTOCOL  L7_PROTO  IN_BYTES  \
Attack                                                                       
Analysis                  230            230       230       230       230   
Backdoor                  217            217       217       217       217   
Benign                 229522         229522    229522    229522    229522   
DoS                       579            579       579       579       579   
Exploits                 3155           3155      3155      3155      3155   
Fuzzers                  2231           2231      2231      2231      2231   
Generic                  1656           1656      1656      1656      1656   
Reconnaissance           1278           1278      1278      1278      1278   
Shellcode                 143            143       143       143       143   
Worms                      16             16        16        16        16   

                IN_PKTS  OUT_BYTES  OUT_PKTS  TCP_FLAGS  CLIENT_TCP_FLAGS  \
Attack                                                                      
Analysis            230        230       230        230               230   
Backdoor            217        217       217        217               217   
Benign           229522     229522    229522     229522            229522   
DoS                 579        579       579        579               579   
Exploits           3155       3155      3155       3155              3155   
Fuzzers            2231       2231      2231       2231              2231   
Generic            1656       1656      1656       1656              1656   
Reconnaissance     1278       1278      1278       1278              1278   
Shellcode           143        143       143        143               143   
Worms                16         16        16         16                16   

                ...  NUM_PKTS_1024_TO_1514_BYTES  TCP_WIN_MAX_IN  \
Attack          ...                                                
Analysis        ...                          230             230   
Backdoor        ...                          217             217   
Benign          ...                       229522          229522   
DoS             ...                          579             579   
Exploits        ...                         3155            3155   
Fuzzers         ...                         2231            2231   
Generic         ...                         1656            1656   
Reconnaissance  ...                         1278            1278   
Shellcode       ...                          143             143   
Worms           ...                           16              16   

                TCP_WIN_MAX_OUT  ICMP_TYPE  ICMP_IPV4_TYPE  DNS_QUERY_ID  \
Attack                                                                     
Analysis                    230        230             230           230   
Backdoor                    217        217             217           217   
Benign                   229522     229522          229522        229522   
DoS                         579        579             579           579   
Exploits                   3155       3155            3155          3155   
Fuzzers                    2231       2231            2231          2231   
Generic                    1656       1656            1656          1656   
Reconnaissance             1278       1278            1278          1278   
Shellcode                   143        143             143           143   
Worms                        16         16              16            16   

                DNS_QUERY_TYPE  DNS_TTL_ANSWER  FTP_COMMAND_RET_CODE   Label  
Attack                                                                        
Analysis                   230             230                   230     230  
Backdoor                   217             217                   217     217  
Benign                  229522          229522                229522  229522  
DoS                        57

In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

FLOW_START_MILLISECONDS  FLOW_END_MILLISECONDS  IPV4_SRC_ADDR  \
Attack                                                                          
Analysis                            123                    123            123   
Backdoor                            345                    345            345   
Benign                           215103                 215103         215103   
DoS                                 505                    505            505   
Exploits                           3882                   3882           3882   
Fuzzers                            2559                   2559           2559   
Generic                             476                    476            476   
Reconnaissance                     1129                   1129           1129   
Shellcode                           159                    159            159   
Worms                                14                     14             14   

                IPV4_DST_ADDR  PROTOCOL  L7_PROTO  IN_BYTES  IN_PKTS  \
Attack                                                                 
Analysis                  123       123       123       123      123   
Backdoor                  345       345       345       345      345   
Benign                 215103    215103    215103    215103   215103   
DoS                       505       505       505       505      505   
Exploits                 3882      3882      3882      3882     3882   
Fuzzers                  2559      2559      2559      2559     2559   
Generic                   476       476       476       476      476   
Reconnaissance           1129      1129      1129      1129     1129   
Shellcode                 159       159       159       159      159   
Worms                      14        14        14        14       14   

                OUT_BYTES  OUT_PKTS  ...  FTP_COMMAND_RET_CODE  \
Attack                               ...                         
Analysis              123       123  ...                   123   
Backdoor              345       345  ...                   345   
Benign             215103    215103  ...                215103   
DoS                   505       505  ...                   505   
Exploits             3882      3882  ...                  3882   
Fuzzers              2559      2559  ...                  2559   
Generic               476       476  ...                   476   
Reconnaissance       1129      1129  ...                  1129   
Shellcode             159       159  ...                   159   
Worms                  14        14  ...                    14   

                SRC_TO_DST_IAT_MIN  SRC_TO_DST_IAT_MAX  SRC_TO_DST_IAT_AVG  \
Attack                                                                       
Analysis                       123                 123                 123   
Backdoor                       345                 345                 345   
Benign                      215103              215103              215103   
DoS                            505                 505                 505   
Exploits                      3882                3882                3882   
Fuzzers                       2559                2559                2559   
Generic                        476                 476                 476   
Reconnaissance                1129                1129                1129   
Shellcode                      159                 159                 159   
Worms                           14                  14                  14   

                SRC_TO_DST_IAT_STDDEV  DST_TO_SRC_IAT_MIN  DST_TO_SRC_IAT_MAX  \
Attack                                                                          
Analysis                          123                 123                 123   
Backdoor                          345                 345                 345   
Benign                         215103              215103              215103   
DoS                               505      

In [19]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

# ADDED
X_test = test_data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y_test = test_data[["Attack", "Label"]]
##

In [21]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [22]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [23]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [24]:
X_train.head()

IPV4_SRC_ADDR  IPV4_DST_ADDR      PROTOCOL      L7_PROTO  \
220614     59.166.0.4  149.171.126.1  4.802078e-09  3.048648e-09   
329844     59.166.0.1  149.171.126.8  9.025364e-09  8.052833e-09   
1093665    59.166.0.3  149.171.126.8  3.068685e-13  8.437668e-13   
1764268    59.166.0.4  149.171.126.0  2.361402e-08  2.106949e-08   
1525176    59.166.0.9  149.171.126.5  2.309362e-10  2.856102e-10   

             IN_BYTES       IN_PKTS  OUT_BYTES      OUT_PKTS     TCP_FLAGS  \
220614   3.994566e-05  7.652425e-07   0.000072  1.071339e-06  1.980229e-09   
329844   1.079121e-04  8.300931e-07   0.000063  8.300931e-07  1.397062e-08   
1093665  1.638983e-07  3.061234e-09   0.000010  7.129838e-09  1.188386e-13   
1764268  7.927293e-05  1.085931e-06   0.000097  1.085931e-06  3.655283e-08   
1525176  1.717880e-05  2.649686e-07   0.000124  2.796890e-07  8.943290e-11   

         CLIENT_TCP_FLAGS  ...  TCP_WIN_MAX_IN  TCP_WIN_MAX_OUT     ICMP_TYPE  \
220614       1.980360e-09  ...    1.329685e-03     1.329685e-03  4.043553e-10   
329844       1.396955e-08  ...    0.000000e+00     0.000000e+00  1.181795e-08   
1093665      1.274923e-13  ...    6.372843e-07     7.080937e-08  4.545956e-14   
1764268      3.655002e-08  ...    0.000000e+00     0.000000e+00  3.092057e-08   
1525176      9.594530e-11  ...    1.598644e-04     1.065762e-04  5.174884e-12   

         ICMP_IPV4_TYPE  DNS_QUERY_ID  DNS_QUERY_TYPE  DNS_TTL_ANSWER  \
220614     4.043553e-10  6.314168e-09    6.314212e-09        0.000000   
329844     1.182599e-08  8.561578e-09    8.561639e-09        0.000000   
1093665    5.191931e-14  4.034960e-13    4.034988e-13        0.000000   
1764268    3.094159e-08  0.000000e+00    3.577436e-10        0.000033   
1525176    5.174884e-12  3.036539e-10    3.036561e-10        0.000000   

         FTP_COMMAND_RET_CODE  \
220614           3.618463e-09   
329844           9.271523e-09   
1093665          4.369547e-13   
1764268          2.425808e-08   
1525176          3.288336e-10   

                                                         h       id  
220614   [4.802078220820267e-09, 3.0486475517277593e-09...   220614  
329844   [9.025363504432674e-09, 8.052833119952719e-09,...   329844  
1093665  [3.0686849809527964e-13, 8.437667985448175e-13...  1093665  
1764268  [2.3614022984160325e-08, 2.1069487814953584e-0...  1764268  
1525176  [2.309362039695701e-10, 2.856102367918858e-10,...  1525176  

[5 rows x 43 columns]

In [25]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [26]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
print(train_g) # 39 nodes, 167318 edges
train_g = train_g.to_directed()
print(train_g) # 39 nodes, 334636 edges (doubled because of directed graph)
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

MultiGraph with 39 nodes and 167318 edges
MultiDiGraph with 39 nodes and 334636 edges


In [27]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [28]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [29]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [30]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [31]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [32]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [33]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [34]:
# Convert to GPU
train_g = train_g.to('cuda')

In [35]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_UNSW.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [36]:
dgi.load_state_dict(torch.load('best_dgi_UNSW.pkl'))

<All keys matched successfully>

In [37]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [38]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [39]:
# Convert to GPU
test_g = test_g.to('cuda')

In [40]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [41]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,) #  334636 x 256
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"])) # 167318 x 40
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,) #  143418 x 256
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"])) # 71709 x 40
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [42]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [43]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [44]:
fuse_test_samples

0         1         2         3         4         5         6  \
0      -0.013382  0.063569 -0.033521  0.063758 -0.033742  0.045809 -0.065304   
1      -0.013382  0.063569 -0.033521  0.063758 -0.033742  0.045809 -0.065304   
2      -0.013382  0.063569 -0.033521  0.063758 -0.033742  0.045809 -0.065304   
3      -0.013382  0.063569 -0.033521  0.063758 -0.033742  0.045809 -0.065304   
4      -0.013382  0.063569 -0.033521  0.063758 -0.033742  0.045809 -0.065304   
...          ...       ...       ...       ...       ...       ...       ...   
448580  0.024852  0.112039 -0.113054  0.131248 -0.062647  0.056990 -0.058300   
448581  0.024852  0.112039 -0.113054  0.131248 -0.062647  0.056990 -0.058300   
448582  0.024852  0.112039 -0.113054  0.131248 -0.062647  0.056990 -0.058300   
448583  0.024852  0.112039 -0.113054  0.131248 -0.062647  0.056990 -0.058300   
448584  0.036142  0.121701 -0.118426  0.125450 -0.064809  0.057661 -0.065066   

               7         8         9  ...  NUM_PKTS_512_TO_1024_BYTES  \
0       0.005479 -0.009770  0.064721  ...                         0.0   
1       0.005479 -0.009770  0.064721  ...                         0.0   
2       0.005479 -0.009770  0.064721  ...                         0.0   
3       0.005479 -0.009770  0.064721  ...                         0.0   
4       0.005479 -0.009770  0.064721  ...                         0.0   
...          ...       ...       ...  ...                         ...   
448580  0.017559  0.027202  0.105126  ...                         0.0   
448581  0.017559  0.027202  0.105126  ...                         0.0   
448582  0.017559  0.027202  0.105126  ...                         0.0   
448583  0.017559  0.027202  0.105126  ...                         0.0   
448584  0.013151  0.027357  0.111058  ...                         0.0   

        NUM_PKTS_1024_TO_1514_BYTES  TCP_WIN_MAX_IN  TCP_WIN_MAX_OUT  \
0                               0.0        0.558214         0.558214   
1                               0.0        0.676099         0.676099   
2                               0.0        0.605015         0.605015   
3                               0.0        0.570583         0.570583   
4                               0.0        0.645465         0.645465   
...                             ...             ...              ...   
448580                          0.0        0.000000         0.000000   
448581                          0.0        0.000000         0.000000   
448582                          0.0        0.000000         0.000000   
448583                          0.0        0.000000         0.000000   
448584                          0.0        0.192917         0.000000   

           ICMP_TYPE  ICMP_IPV4_TYPE  DNS_QUERY_ID  DNS_QUERY_TYPE  \
0       4.286919e-06        0.000005  1.405707e-06    1.405717e-06   
1       5.192235e-06        0.000006  1.702565e-06    1.702577e-06   
2       5.556507e-06        0.000006  1.523560e-06    1.523571e-06   
3       9.891732e-06        0.000011  1.436853e-06    1.436863e-06   
4       3.268869e-06        0.000003  1.625423e-06    1.625435e-06   
...              ...             ...           ...             ...   
448580  2.382185e-07        0.000002  2.471654e-07    2.471672e-07   
448581  2.362368e-07        0.000002  2.451092e-07    2.451109e-07   
448582  2.402778e-07        0.000002  2.493021e-07    2.493038e-07   
448583  2.444432e-07        0.000002  2.536239e-07    2.536257e-07   
448584  2.102052e-06        0.000002  1.362841e-06    1.362851e-06   

        DNS_TTL_ANSWER  FTP_COMMAND_RET_CODE  
0                  0.0          1.522271e-06  
1                  0.0          1.843746e-06  
2                  0.0          1.649897e-06  
3                  0.0          1.556000e-06  
4                  0.0          1.760207e-06  
...                ...                   ...  
448580             0.0          2.676609e-07  
448581             0.0          2.654342e-07  
448582             0.0          2.699747e-07  
4

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [46]:
from pyod.models.cblof import CBLOF
n_est = [5,6,7,9,10] # can't be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [02:29<00:00,  4.97s/it]

{'n_estimators': 7, 'con': 0.01}
0.7624217740035879
              precision    recall  f1-score   support

           0     0.9804    0.9808    0.9806    430201
           1     0.5464    0.5420    0.5442     18384

    accuracy                         0.9628    448585
   macro avg     0.7634    0.7614    0.7624    448585
weighted avg     0.9627    0.9628    0.9627    448585



In [47]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [02:32<00:00,  5.09s/it]

{'n_estimators': 9, 'con': 0.01}
0.7589400972754892
              precision    recall  f1-score   support

           0     0.9783    0.9857    0.9820    430201
           1     0.5931    0.4888    0.5359     18384

    accuracy                         0.9653    448585
   macro avg     0.7857    0.7372    0.7589    448585
weighted avg     0.9625    0.9653    0.9637    448585



In [48]:
# HBOS  Embeddings+Raw (Multimodal)

In [49]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [50]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [03:06<00:00,  5.19s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [51]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [03:09<00:00,  5.26s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [52]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [53]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:16<00:00,  3.79s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [54]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:19<00:00,  3.87s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [55]:
##  IF  Emb+Raw

In [56]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [57]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:01<00:27,  1.20s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:02<00:26,  1.21s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:03<00:25,  1.19s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%

{'n_estimators': 20, 'con': 0.01}
0.9990514033671924
              precision    recall  f1-score   support

           0     1.0000    0.9999    0.9999    430201
           1     0.9966    0.9998    0.9982     18384

    accuracy                         0.9999    448585
   macro avg     0.9983    0.9998    0.9991    448585
weighted avg     0.9999    0.9999    0.9999    448585



In [58]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:37<00:00,  1.55s/it]

{'n_estimators': 50, 'con': 0.001}
0.9986963989946824
              precision    recall  f1-score   support

           0     0.9999    0.9999    0.9999    430201
           1     0.9967    0.9983    0.9975     18384

    accuracy                         0.9998    448585
   macro avg     0.9983    0.9991    0.9987    448585
weighted avg     0.9998    0.9998    0.9998    448585

